In [ ]:
import sys
sys.path.append('../../') # or just install the module
sys.path.append('../../../fuzzy-tools') # or just install the module
sys.path.append('../../../astro-lightcurves-handler') # or just install the module

In [ ]:
from fuzzytools.files import search_for_filedirs
from lchandler import _C

root_folder = '../../../surveys-save'
filedirs = search_for_filedirs(root_folder, fext=_C.EXT_SPLIT_LIGHTCURVE)

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from fuzzytools.files import load_pickle, save_pickle
from fuzzytools.files import get_dict_from_filedir
from lchandler import _C

def load_lcdataset(filename):
    assert filename.split('.')[-1]==_C.EXT_SPLIT_LIGHTCURVE
    return load_pickle(filename)

filedir = '../../../surveys-save/alerceZTFv7.1/survey=alerceZTFv7.1°bands=gr°mode=onlySNe.splcds'

filedict = get_dict_from_filedir(filedir)
root_folder = filedict['*rootdir*']
cfilename = filedict['*cfilename*']
survey = filedict['survey']
lcdataset = load_lcdataset(filedir)
print(lcdataset['raw'].keys())
print(lcdataset['raw'].get_random_lcobj(False).keys())
print(lcdataset)

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from synthsne.generators.synthetic_curves import get_syn_sne_generator
from synthsne.plots.lc import plot_synthetic_samples
from synthsne import _C

set_name = 'train'

save_rootdir = f'../../save/{survey}/{cfilename}/{set_name}'
samplers = load_pickle(f'{save_rootdir}/samplers.{_C.EXT_SAMPLER}')
obse_sampler_bdict = samplers['obse_sampler_bdict']
length_sampler_bdict = samplers['length_sampler_bdict']
    
method = 'uniformprior'
method = 'curvefit'
method = 'mcmc'
#method = 'linear'
#method = 'bspline'

synthetic_samples_per_curve = 32

lcset = lcdataset[set_name]
lcobj_names = [
'ZTF18acqyvag',
'ZTF18aaqvuhl',
'ZTF20aamuqwn',
'ZTF18adasisj',
'ZTF20aavgmli',
'ZTF19aaqcqkv',
'ZTF19aadnxat',
'ZTF19aaripqw',
'ZTF18aaqkdwf',
]
for lcobj_name in lcobj_names:
    if not lcobj_name in lcset.get_lcobj_names():
        continue
    lcobj = lcset[lcobj_name]
    band_names = lcset.band_names
    class_names = lcset.class_names
    gen_kwargs = {
        #'uses_new_bounds':False,
        'uses_new_bounds':True,
    }
    generator_class = get_syn_sne_generator(method)
    sne_generator = generator_class(lcobj, class_names, band_names, obse_sampler_bdict, length_sampler_bdict, **gen_kwargs)
    new_lcobjs, new_smooth_lcojbs, trace_bdict, segs = sne_generator.sample_curves(synthetic_samples_per_curve)
    plot_kwargs = {
        'trace_bdict':trace_bdict,
        #'save_filedir':f'../save/test_{lcobj_name}.png',
        'save_filedir':None,
    }
    plot_synthetic_samples(lcset, set_name, method, lcobj_name, new_lcobjs, new_smooth_lcojbs, **plot_kwargs)
    plt.show()
    #break